In [ ]:
import google.colab as colab
colab.drive.mount('/content/drive')

In [ ]:
import os
from PIL import Image, ImageSequence

# Define the directory to save frames on Google Drive
output_dir = '/content/drive/MyDrive/Dataset/child/action/frames'
os.makedirs(output_dir, exist_ok=True)

# Function to extract and save frames from a GIF
def save_frames_from_gif(gif_path, action_label):
    # Create an action-specific folder to save the frames
    action_dir = os.path.join(output_dir, action_label)
    os.makedirs(action_dir, exist_ok=True)

    img = Image.open(gif_path)
    for idx, frame in enumerate(ImageSequence.Iterator(img)):
        frame_rgb = frame.convert('RGB')  # Convert to RGB
        frame_filename = f"{action_label}_frame_{idx+33}.jpg"
        frame_path = os.path.join(action_dir, frame_filename)
        frame_rgb.save(frame_path)
        print(f"Saved: {frame_path}")

# Paths and labels for the GIF files
actions = ['kick', 'punch', 'running', 'squat', 'stand', 'wave']
gif_paths = [

]

# Extract frames for each GIF and save in respective action folders
for gif_path, action_label in zip(gif_paths, actions):
    save_frames_from_gif(gif_path, action_label)

print("All frames extracted and saved successfully.")


# Count the number of images in each action folder
print("\nImage Counts in Each Folder:")
for action_label in actions:
    action_dir = os.path.join(output_dir, action_label)
    num_images = len([img for img in os.listdir(action_dir) if img.endswith('.jpg')])
    print(f"{action_label}: {num_images} images")


In [ ]:
!pip install --upgrade tensorflow tensorflow_hub


In [ ]:
import os
import json
import numpy as np
from PIL import Image
import tensorflow as tf

# Load the MoveNet model (TensorFlow Hub)
movenet_model = tf.saved_model.load('https://tfhub.dev/google/movenet/singlepose/lightning/4')

# Directory for saving keypoints
keypoints_dir = '/content/drive/MyDrive/Dataset/child/action/pose_keypoints'
os.makedirs(keypoints_dir, exist_ok=True)

# Function to detect keypoints using MoveNet
def detect_pose(image):
    # Convert PIL Image to NumPy array
    image_rgb = np.array(image)
    image_rgb = tf.convert_to_tensor(image_rgb, dtype=tf.float32)

    # Resize and normalize the image (MoveNet requires 192x192 input size)
    input_image = tf.image.resize(image_rgb, (192, 192))
    input_image = input_image / 255.0  # Normalize image

    # Run inference with MoveNet model
    input_image = input_image[tf.newaxis,...]  # Add batch dimension
    outputs = movenet_model(input_image)

    # Extract keypoints (pose landmarks) from MoveNet's output
    keypoints = outputs['output_0'].numpy()
    keypoints = keypoints.squeeze()  # Remove batch dimension

    # Extract keypoints for x, y, confidence (visibility)
    keypoints = keypoints[:, :3]  # We are only interested in x, y, and confidence
    return keypoints

# Function to process and save keypoints for all frames in action folders
def save_keypoints_for_action(action_folder, action_label):
    keypoints_data = []

    # Retrieve all image files from the action-specific folder
    for filename in sorted(os.listdir(action_folder)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Supported image formats
            image_path = os.path.join(action_folder, filename)
            frame = Image.open(image_path).convert('RGB')  # Convert to RGB

            # Detect pose keypoints
            keypoints = detect_pose(frame)
            keypoints_data.append(keypoints.tolist())

    # Save the keypoints with the action label
    data = {
        'label': action_label,
        'keypoints': keypoints_data
    }

    # Save data as JSON
    json_path = os.path.join(keypoints_dir, f"{action_label}_keypoints.json")
    with open(json_path, 'w') as f:
        json.dump(data, f)

# Define the action directories
actions = ['kick', 'punch', 'squat', 'stand', 'wave', 'running', 'sit', 'fall']
action_folders = [
    '/content/drive/MyDrive/Dataset/child/action/frames/kick',
    '/content/drive/MyDrive/Dataset/child/action/frames/punch',
    '/content/drive/MyDrive/Dataset/child/action/frames/squat',
    '/content/drive/MyDrive/Dataset/child/action/frames/stand',
    '/content/drive/MyDrive/Dataset/child/action/frames/wave',
    '/content/drive/MyDrive/Dataset/child/action/frames/running',
    '/content/drive/MyDrive/Dataset/child/action/frames/fall',
    '/content/drive/MyDrive/Dataset/child/action/frames/sit'
]

# Process each action folder and generate keypoints JSON
for action_folder, action_label in zip(action_folders, actions):
    save_keypoints_for_action(action_folder, action_label)

print("Keypoint extraction completed for all actions.")


In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np
import os
from PIL import Image

# Function to visualize keypoints on an image (modified for MediaPipe)
def visualize_keypoints(image, keypoints):
    plt.imshow(image)
    image_width, image_height = image.size
    for i, keypoint in enumerate(keypoints):
        x, y, z, visibility = keypoint
        if visibility > 0.5:  # Show only keypoints with visibility > 0.5
            plt.scatter(x * image_width, y * image_height, s=10, c='red', marker='o')
            plt.text(x * image_width, y * image_height, str(i), color="blue", fontsize=8)
    plt.axis('off')

# Directory paths
keypoints_dir = '/content/drive/MyDrive/Dataset/child/action/pose_keypoints'
frame_folders = {
    'kick': '/content/drive/MyDrive/Dataset/child/action/frames/kick',
    'punch': '/content/drive/MyDrive/Dataset/child/action/frames/punch',
    'squat': '/content/drive/MyDrive/Dataset/child/action/frames/squat',
    'stand': '/content/drive/MyDrive/Dataset/child/action/frames/stand',
    'wave': '/content/drive/MyDrive/Dataset/child/action/frames/wave',
    'running': '/content/drive/MyDrive/Dataset/child/action/frames/running',
    'fall': '/content/drive/MyDrive/Dataset/child/action/frames/fall',
    'sit':  '/content/drive/MyDrive/Dataset/child/action/frames/sit'
}

# Function to load and visualize keypoints for each frame in a given class folder
def visualize_keypoints_for_class(class_name, frame_folder, keypoints_path):
    # Load keypoints JSON data
    with open(keypoints_path, 'r') as f:
        keypoints_data = json.load(f)

    # Iterate through frames in the directory
    frame_files = sorted([file for file in os.listdir(frame_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))])

    # Display up to 10 frames with keypoints
    for i, frame_file in enumerate(frame_files[:10]):
        frame_path = os.path.join(frame_folder, frame_file)
        frame = Image.open(frame_path).convert('RGB')

        # Get keypoints for the current frame
        keypoints = keypoints_data['keypoints'][i]

        # Visualize keypoints on the current frame
        plt.figure(figsize=(5, 5))
        visualize_keypoints(frame, keypoints)
        plt.title(f"Class: {class_name} - Frame {i+1}")
        plt.show()

# Visualize keypoints for each action class
for class_name, frame_folder in frame_folders.items():
    keypoints_path = os.path.join(keypoints_dir, f"{class_name}_keypoints.json")
    visualize_keypoints_for_class(class_name, frame_folder, keypoints_path)


In [ ]:
import os
import json
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Function to create subsequences from a sequence
def create_subsequences(sequence, window_size, step_size):
    subsequences = []
    for start in range(0, len(sequence) - window_size + 1, step_size):
        end = start + window_size
        subsequences.append(sequence[start:end])
    return subsequences

# Load keypoints from JSON files and create subsequences
def load_keypoints(keypoints_dir, window_size=30, step_size=5):
    X = []
    y = []
    for filename in os.listdir(keypoints_dir):
        if filename.endswith('.json'):
            filepath = os.path.join(keypoints_dir, filename)
            with open(filepath, 'r') as f:
                data = json.load(f)
                keypoints = data['keypoints']
                label = data['label']

                # Flatten the keypoints for each frame
                flattened_keypoints = [np.array(kp).flatten() for kp in keypoints]

                # Create subsequences
                subsequences = create_subsequences(flattened_keypoints, window_size, step_size)
                X.extend(subsequences)
                y.extend([label] * len(subsequences))

    # Convert X and y to arrays
    X = np.array(X)
    y = np.array(y)

    return X, y

# Load keypoints data
X, y = load_keypoints(keypoints_dir)

# Convert labels to numerical values
actions = ['kick', 'punch', 'squat', 'stand', 'wave', 'running', 'sit', 'fall']
action_map = {action: i for i, action in enumerate(actions)}
y_numeric = np.array([action_map[label] for label in y])

# One-hot encode the labels
y_encoded = to_categorical(y_numeric, num_classes=len(actions))

# Split the data into training and test sets using stratified sampling
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_numeric
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, GRU, Bidirectional

# Build LSTM model
model = Sequential()

model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(actions), activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Save the trained model
model.save('/content/drive/MyDrive/Dataset/child/action/action_recognition_model.h5')


In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Predict the action for a new sequence of keypoints
new_keypoints = X_test[0].reshape(1, X_test.shape[1], X_test.shape[2])
predicted_action = model.predict(new_keypoints)
predicted_label = actions[np.argmax(predicted_action)]
print(f"Predicted Action: {predicted_label}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Predict actions for all test data
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

# Confusion matrix
cm = confusion_matrix(y_true_labels, y_pred_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=actions, yticklabels=actions)
plt.xlabel('Predicted Action')
plt.ylabel('True Action')
plt.title('Confusion Matrix for Action Recognition')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')  # Place legend in the lower right corner
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')  # Place legend in the upper right corner
plt.show()


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image

# Load the trained model
model = load_model('/content/drive/MyDrive/Dataset/child/action/action_recognition_model.h5')

# Load the MoveNet model to extract keypoints
import tensorflow_hub as hub
movenet = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")

# Function to detect keypoints for a single image
def detect_pose(image):
    input_image = tf.image.resize_with_pad(np.expand_dims(image, axis=0), 256, 256)
    input_image = tf.cast(input_image, dtype=tf.int32)
    keypoints_with_scores = movenet.signatures['serving_default'](input_image)
    keypoints = keypoints_with_scores['output_0'].numpy()[0, 0, :, :]
    return keypoints[:, :2]  # Extract only x and y coordinates

# Load and preprocess a test image
image_path = '/content/drive/MyDrive/Dataset/child/action/frames/running/running_frame_0.jpg'  # Replace with your test image path
image = Image.open(image_path).convert('RGB')
image_np = np.array(image)

# Detect keypoints
keypoints = detect_pose(image_np)

# Prepare the keypoints for model input
# Ensure that keypoints are reshaped to match the input shape of the model
# If your model expects (timesteps, features), add a new axis if necessary
keypoints = np.expand_dims(keypoints, axis=0)  # Add batch dimension
keypoints = np.expand_dims(keypoints, axis=0)  # Add time dimension if needed

# Predict the action
predictions = model.predict(keypoints)
predicted_class = np.argmax(predictions)

# Map the prediction to class label
action_labels = ['kick', 'punch', 'squat', 'stand', 'wave', 'running', 'fall', 'sit']  # Update with your actual labels
predicted_label = action_labels[predicted_class]

print(f"The model predicts this action as: {predicted_label}")


In [ ]:
# Install MediaPipe if not already installed
!pip install mediapipe

# Import necessary libraries
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:

# Load the image
image_path = '/content/drive/MyDrive/Dataset/child/action/frames/stand/stand_frame_102.jpg'  # Replace with your image path
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils  # For drawing keypoints and skeleton
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Process the image and extract keypoints
results = pose.process(image_rgb)

# Create a copy of the image to draw on
image_with_keypoints = image_rgb.copy()

# Check if any landmarks are detected
if not results.pose_landmarks:
    print("No pose landmarks detected.")
    keypoints = np.zeros((33, 3))  # MediaPipe provides 33 landmarks
else:
    # Draw pose landmarks on the image
    mp_drawing.draw_landmarks(
        image_with_keypoints,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))

    # Extract keypoints
    landmarks = results.pose_landmarks.landmark
    keypoints = []
    for landmark in landmarks:
        x = landmark.x  # Normalized x-coordinate
        y = landmark.y  # Normalized y-coordinate
        visibility = landmark.visibility  # Confidence score
        keypoints.append([y, x, visibility])
    keypoints = np.array(keypoints)

# Flatten keypoints
keypoints_flatten = keypoints.flatten()

# Define window size (should match the window_size used during training)
window_size = 30

# Create a sequence by replicating the keypoints
sequence = np.array([keypoints_flatten] * window_size)

# Reshape to match the input shape expected by the model
sequence = np.expand_dims(sequence, axis=0)  # Shape: (1, window_size, num_features)

# Load the trained model
model = load_model('/content/drive/MyDrive/Dataset/child/action/action_recognition_model.h5')

# Get the prediction probabilities
prediction = model.predict(sequence)

# Get the action with the highest probability
predicted_action_index = np.argmax(prediction[0])

# Define the action labels (should match the labels used during training)
actions = ['kick', 'punch', 'squat', 'stand', 'wave', 'running', 'sit', 'fall']

# Get the predicted action label
predicted_action = actions[predicted_action_index]

# Get the confidence score of the prediction
confidence_score = prediction[0][predicted_action_index]

# Choose font and position for the text
font = cv2.FONT_HERSHEY_SIMPLEX
position = (10, 30)  # Adjust as needed
font_scale = 1
font_color = (255, 0, 0)  # Blue color in BGR
thickness = 2

# Prepare the text
text = f"Predicted Action: {predicted_action} ({confidence_score*100:.2f}%)"

# Convert image back to BGR for OpenCV
image_with_keypoints_bgr = cv2.cvtColor(image_with_keypoints, cv2.COLOR_RGB2BGR)

# Put text on the image
cv2.putText(image_with_keypoints_bgr, text, position, font, font_scale, font_color, thickness, cv2.LINE_AA)

# Convert image back to RGB for Matplotlib
final_image = cv2.cvtColor(image_with_keypoints_bgr, cv2.COLOR_BGR2RGB)

# Display the image
plt.figure(figsize=(10, 8))
plt.imshow(final_image)
plt.axis('off')
plt.title('Action Recognition Result')
plt.show()
